<a href="https://colab.research.google.com/github/ArthurLazdin/Machine-Learning/blob/main/Notebooks/Hotels_Bookingt_Exercise_Optuna.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Заселится клиент или не заселится? Часть третья. optuna


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from datetime import datetime

In [ ]:
hotel_resrv = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Machine Learning/Data/Hotel Reservations.csv')


## Описание набора.
[Набор](https://www.kaggle.com/datasets/ahsan81/hotel-reservations-classification-dataset) представляет собой информацию о бронировании отелей в онлайн режиме и предназначен для обучения модели-классификатора, которая будет прогнозировать заселится ли клиент или откажется от бронирования.
Набор включает следующие переменные:

* Booking_ID: unique identifier of each booking
* no_of_adults: Number of adults
* no_of_children: Number of Children
* no_of_weekend_nights: Number of weekend nights (Saturday or Sunday) the guest stayed or booked to stay at the hotel
* no_of_week_nights: Number of week nights (Monday to Friday) the guest stayed or booked to stay at the hotel
* type_of_meal_plan: Type of meal plan booked by the customer:
* required_car_parking_space: Does the customer require a car parking space? (0 - No, 1- Yes)
* room_type_reserved: Type of room reserved by the customer. The values are ciphered (encoded) by INN Hotels.
* lead_time: Number of days between the date of booking and the arrival date
* arrival_year: Year of arrival date
* arrival_month: Month of arrival date
* arrival_date: Date of the month
* market_segment_type: Market segment designation.
* repeated_guest: Is the customer a repeated guest? (0 - No, 1- Yes)
* no_of_previous_cancellations: Number of previous bookings that were canceled by the customer prior to the current booking
* no_of_previous_bookings_not_canceled: Number of previous bookings not canceled by the customer prior to the current booking
* avg_price_per_room: Average price per day of the reservation; prices of the rooms are dynamic. (in euros)
* no_of_special_requests: Total number of special requests made by the customer (e.g. high floor, view from the room, etc)
* booking_status: Flag indicating if the booking was canceled or not.
Целевой переменной является booking_status.


In [ ]:
hotel_resrv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36275 entries, 0 to 36274
Data columns (total 19 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Booking_ID                            36275 non-null  object 
 1   no_of_adults                          36275 non-null  int64  
 2   no_of_children                        36275 non-null  int64  
 3   no_of_weekend_nights                  36275 non-null  int64  
 4   no_of_week_nights                     36275 non-null  int64  
 5   type_of_meal_plan                     36275 non-null  object 
 6   required_car_parking_space            36275 non-null  int64  
 7   room_type_reserved                    36275 non-null  object 
 8   lead_time                             36275 non-null  int64  
 9   arrival_year                          36275 non-null  int64  
 10  arrival_month                         36275 non-null  int64  
 11  arrival_date   

В наборе нет пропусков. Это не может не радовать. Посмотрим на примерное содержимое переменных.

In [ ]:
hotel_resrv.head()


,Booking_ID,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,type_of_meal_plan,required_car_parking_space,room_type_reserved,lead_time,arrival_year,arrival_month,arrival_date,market_segment_type,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,avg_price_per_room,no_of_special_requests,booking_status
0,INN00001,2,0,1,2,Meal Plan 1,0,Room_Type 1,224,2017,10,2,Offline,0,0,0,65.00,0,Not_Canceled
1,INN00002,2,0,2,3,Not Selected,0,Room_Type 1,5,2018,11,6,Online,0,0,0,106.68,1,Not_Canceled
2,INN00003,1,0,2,1,Meal Plan 1,0,Room_Type 1,1,2018,2,28,Online,0,0,0,60.00,0,Canceled
3,INN00004,2,0,0,2,Meal Plan 1,0,Room_Type 1,211,2018,5,20,Online,0,0,0,100.00,0,Canceled
4,INN00005,2,0,1,1,Not Selected,0,Room_Type 1,48,2018,4,11,Online,0,0,0,94.50,0,Canceled


In [ ]:
hotel_resrv.describe()

,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,required_car_parking_space,lead_time,arrival_year,arrival_month,arrival_date,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,avg_price_per_room,no_of_special_requests
count,36275.000000,36275.000000,36275.000000,36275.000000,36275.000000,36275.000000,36275.000000,36275.000000,36275.000000,36275.000000,36275.000000,36275.000000,36275.000000,36275.000000
mean,1.844962,0.105279,0.810724,2.204300,0.030986,85.232557,2017.820427,7.423653,15.596995,0.025637,0.023349,0.153411,103.423539,0.619655
std,0.518715,0.402648,0.870644,1.410905,0.173281,85.930817,0.383836,3.069894,8.740447,0.158053,0.368331,1.754171,35.089424,0.786236
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2017.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.000000,0.000000,0.000000,1.000000,0.000000,17.000000,2018.000000,5.000000,8.000000,0.000000,0.000000,0.000000,80.300000,0.000000
50%,2.000000,0.000000,1.000000,2.000000,0.000000,57.000000,2018.000000,8.000000,16.000000,0.000000,0.000000,0.000000,99.450000,0.000000
75%,2.000000,0.000000,2.000000,3.000000,0.000000,126.000000,2018.000000,10.000000,23.000000,0.000000,0.000000,0.000000,120.000000,1.000000
max,4.000000,10.000000,7.000000,17.000000,1.000000,443.000000,2018.000000,12.000000,31.000000,1.000000,13.000000,58.000000,540.000000,5.000000


##Краткий обзор. 

Все соображения по выбору переменных сохранились из предыдущего ноутбука, поэтому повторяющиеся тексты и иллюстрации можно удалить.


In [ ]:
# преобразуем целевую переменную
booking_st  = {'Canceled': 0, 'Not_Canceled': 1}
hotel_resrv['booking_status'] = hotel_resrv['booking_status'].map(booking_st)

# удалим идентификаторы
hotel_resrv.drop(['Booking_ID'], axis=1, inplace=True)

In [ ]:
#преобразуем переменную room_type_reserved
room_type = hotel_resrv['room_type_reserved'].unique()
temp_list = np.arange(len(room_type))

room_tipe_dict = dict(zip(room_type, temp_list))
hotel_resrv['room_type_reserved'] = hotel_resrv['room_type_reserved'].map(room_tipe_dict)


In [ ]:
segment_type = hotel_resrv['market_segment_type'].unique()
temp_list = np.arange(len(segment_type))

segment_tipe_dict = dict(zip(segment_type, temp_list))
hotel_resrv['market_segment_type'] = hotel_resrv['market_segment_type'].map(segment_tipe_dict)

In [ ]:
meal_plan = hotel_resrv['type_of_meal_plan'].unique()
temp_list = np.arange(len(meal_plan))

meal_plan_dict = dict(zip(meal_plan, temp_list))
hotel_resrv['type_of_meal_plan'] = hotel_resrv['type_of_meal_plan'].map(meal_plan_dict)

In [ ]:
# удаляем исходные переменные с датами "вразбивку"
hotel_resrv.drop([ 'arrival_year','arrival_month','arrival_date'], axis=1, inplace=True)


##EDA
Рассмотрим влияние значений no_of_previous_cancellations	и no_of_previous_bookings_not_canceled на принятие решения о заселении.

In [ ]:
no_cancel = len(hotel_resrv[hotel_resrv['no_of_previous_bookings_not_canceled'] == 0 ])
cancel = len(hotel_resrv[hotel_resrv['no_of_previous_cancellations'] == 0 ])
total = len(hotel_resrv)
print('No cancel = ' , (no_cancel / total)*100, '%   Cancel = ', (cancel/total)*100 ,'%')

No cancel =  97.76154376292212 %   Cancel =  99.06822880771881 %


Видно, что пользы от них исчезающе мало.
Поэтому мы удалим эти столбцы из датасета.

In [ ]:
hotel_resrv.drop([ 'no_of_previous_bookings_not_canceled', 'no_of_previous_cancellations'], axis=1, inplace=True)

### Построим корреляционную матрицу.
Интересно, хотя два самых коррелируемых значения - это число детей и число комнат. 
Подробно займемся анализом матрицы в ходе работы над третим заданием.

In [ ]:
import plotly.figure_factory as ff
corrs = hotel_resrv.corr()
figure = ff.create_annotated_heatmap(
    z=corrs.values,
    x=list(corrs.columns),
    y=list(corrs.index),
    annotation_text=corrs.round(2).values,
    showscale=True)
figure.show()

Оставим данные о количестве забронированных ночей, как есть. Определенная корреляция с этими переменными есть. 


# Строим модель.

In [ ]:
feature_names = list(hotel_resrv).copy()
feature_names.remove('booking_status')

# выделчем отдельно целевую переменную
target = hotel_resrv['booking_status']

# создаем датасет из выбранных переменных кроме целевой
temp_hotel_res = hotel_resrv
temp_hotel_res.drop(['booking_status'], axis=1, inplace=True)

X = pd.DataFrame(temp_hotel_res, columns = feature_names) 
y = target


In [ ]:
from sklearn.model_selection import train_test_split

#Создаем три набора данных: тренировочный, тестовый и валидационный. В пропорции 60:20:20.
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size = 0.4, 
                                                    random_state = 42)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test,
                                                test_size=0.5, 
                                                random_state=42)


### Нормализуем данные.
Каждый датасет нормализуем по-отдельности, чтобы не переносить усреднения из набора в набор.

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

scaled_temp = scaler.fit_transform(X_train).copy()
X_train = pd.DataFrame(scaled_temp, columns = feature_names) 

scaled_temp = scaler.fit_transform(X_test).copy()
X_test = pd.DataFrame(scaled_temp, columns = feature_names) 

scaled_temp = scaler.fit_transform(X_val).copy()
X_val = pd.DataFrame(scaled_temp, columns = feature_names) 

X_test.head()


,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,type_of_meal_plan,required_car_parking_space,room_type_reserved,lead_time,market_segment_type,repeated_guest,avg_price_per_room,no_of_special_requests
0,0.301832,-0.261465,-0.932787,-0.827733,-0.507096,-0.178491,-0.429013,-0.767799,0.301847,-0.17095,0.019974,-0.795079
1,-1.600679,-0.261465,-0.932787,-0.827733,-0.507096,-0.178491,0.850268,-0.944088,3.397110,-0.17095,-0.233073,-0.795079
2,-1.600679,-0.261465,0.212090,-0.134051,-0.507096,-0.178491,-0.429013,-0.944088,1.849478,-0.17095,-0.682934,-0.795079
3,0.301832,-0.261465,0.212090,-0.134051,1.084159,-0.178491,-0.429013,-0.050890,0.301847,-0.17095,-0.677311,1.734877
4,-1.600679,-0.261465,2.501845,3.334357,-0.507096,-0.178491,-0.429013,-0.850067,0.301847,-0.17095,-0.741135,0.469899


## Выбор модели
В качестве модели выберем случайный лес. Он же Random Forest. 
В Интернете у него хорошая репутация.

In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score


## Кросс-валидация. (из предыдущего ноутбука)
Для предыдущего задания была выбрана модель на основе метода логистической регрессии. Сравним ее с методом случайного леса.

In [ ]:
from sklearn.linear_model import LogisticRegression
model1 = LogisticRegression()
scores1 = cross_val_score(model1, X_train, y_train.values.ravel(), cv=5)
print('Logistic Regression',scores1)

model = RandomForestClassifier()
scores = cross_val_score(model, X_train, y_train.values.ravel(), cv=5)
print('Random Forest',scores)



Logistic Regression [0.79600276 0.77670572 0.78704342 0.79186768 0.78313807]
Random Forest [0.88766368 0.87870434 0.88375833 0.88513669 0.8731909 ]


Видно, что метод случайного леса работает явно лучше. На нем и остановимся

## Настройка гиперпараметров используя optuna.

In [ ]:
!pip install optuna > None

In [ ]:
import optuna
from sklearn.svm import SVC

In [ ]:
def objective(trial):
    classifier_name = trial.suggest_categorical("classifier", ["SVC", "RandomForest"])
    if classifier_name == "SVC":
      svc_c = trial.suggest_float("svc_c", 0.00001, 10000, log=True)
      classifier_obj = SVC(C=svc_c, gamma="auto")
    else:
      rf_max_depth = trial.suggest_int("rf_max_depth", 10, 40, log=True)  #немного изменил границы поиска
      classifier_obj = RandomForestClassifier(max_depth=rf_max_depth, n_estimators=30)
    #используем валидационные датасеты
    score = cross_val_score(classifier_obj, X_val, y_val, n_jobs=-1, cv=3)
    accuracy = score.mean()
    return accuracy


In [ ]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50)

[I 2023-02-06 10:50:54,507] A new study created in memory with name: no-name-7d44a321-cc6e-4e7e-acf9-49f31875b304
[I 2023-02-06 10:50:58,871] Trial 0 finished with value: 0.8518258461383453 and parameters: {'classifier': 'RandomForest', 'rf_max_depth': 10}. Best is trial 0 with value: 0.8518258461383453.
[I 2023-02-06 10:51:06,163] Trial 1 finished with value: 0.735079435582176 and parameters: {'classifier': 'SVC', 'svc_c': 0.012883294125532662}. Best is trial 0 with value: 0.8518258461383453.
[I 2023-02-06 10:51:06,976] Trial 2 finished with value: 0.8589935070818934 and parameters: {'classifier': 'RandomForest', 'rf_max_depth': 11}. Best is trial 2 with value: 0.8589935070818934.
[I 2023-02-06 10:51:13,295] Trial 3 finished with value: 0.6662990913881045 and parameters: {'classifier': 'SVC', 'svc_c': 2.1571522579371005e-05}. Best is trial 2 with value: 0.8589935070818934.
[I 2023-02-06 10:51:17,225] Trial 4 finished with value: 0.6662990913881045 and parameters: {'classifier': 'SVC',

In [ ]:
print(study.best_trial)

FrozenTrial(number=13, state=TrialState.COMPLETE, values=[0.8635421514243742], datetime_start=datetime.datetime(2023, 2, 6, 10, 53, 24, 907884), datetime_complete=datetime.datetime(2023, 2, 6, 10, 53, 25, 427884), params={'classifier': 'RandomForest', 'rf_max_depth': 16}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'classifier': CategoricalDistribution(choices=('SVC', 'RandomForest')), 'rf_max_depth': IntDistribution(high=40, log=True, low=10, step=1)}, trial_id=13, value=None)


In [ ]:
from optuna.visualization import plot_optimization_history
plot_optimization_history(study)

## Обучение модели

In [ ]:
%%time
model_opt = RandomForestClassifier(n_estimators = 30, max_depth = 16)
model_opt.fit(X_train, y_train.values.ravel())


CPU times: user 549 ms, sys: 5.7 ms, total: 555 ms
Wall time: 558 ms


RandomForestClassifier(max_depth=16, n_estimators=30)

## Оценка модели. 
И работу моделей на валидационных наборах.

In [ ]:
y_pred = model_opt.predict(X_val)
accuracy = round(accuracy_score(y_val, y_pred), 3)
precision = round(precision_score(y_val, y_pred), 3)
recall = round(recall_score(y_val, y_pred), 3)

print('Max Depth: {} || Estimators: {} || Accuracy: {} || Precision: {} || Recall: {}'.format(model_opt.max_depth,
                                                                                                model_opt.n_estimators,
                                                                                                accuracy,
                                                                                                precision,
                                                                                                recall))

Max Depth: 16 || Estimators: 30 || Accuracy: 0.877 || Precision: 0.886 || Recall: 0.935


Теперь запустим модель на тестовом наборе. 

In [ ]:
y_pred = model_opt.predict(X_test)
accuracy = round(accuracy_score(y_test, y_pred), 6)
precision = round(precision_score(y_test, y_pred), 6)
recall = round(recall_score(y_test, y_pred), 6)

print('Max Depth: {} || Estimators: {} || Accuracy: {} || Precision: {} || Recall: {}'.format(model_opt.max_depth,
                                                                                                model_opt.n_estimators,
                                                                                                accuracy,
                                                                                                precision,
                                                                                                recall))



Max Depth: 16 || Estimators: 30 || Accuracy: 0.874294 || Precision: 0.884971 || Recall: 0.935819


In [ ]:
from sklearn.metrics import confusion_matrix
model_matrix = confusion_matrix(y_test, y_pred, labels = [1,0])
#вспомним матрицу ошибок и выведем ее
# для удобства создадим датафрейм
model_matrix_df = pd.DataFrame(model_matrix)
print(model_matrix_df)

      0     1
0  4593   315
1   597  1750
